# Model Evaluation for NN Ensembles

Purpose: to automate the evaluation of models to check their suitability for use in an ensemble

The main insight here is that we want to pick *Specialist* models who score highly (total correct aka True Positives) and high precision for the specific category they specialize in.  We care that a) we have a high number of correct predictions and b) of all our predictions, the highest number of them are correct.  In order to evaluate we create a simple index,

$$
I = Precision  *  TP = \frac{(TP)^2}{TP+FP}
$$



In [ ]:
RUN_NAMES = []  # list of run names (of models) to load
EPS2EVAL = []  # list of int tuples forming the low and high epoch numbers to range thru


class model_score():
'''Simple structure to keep score data for each model/epoch'''    
    init (run_name, epoch_num, clsf_rpt):
        self.run_name = run_name
        self.epoch_num = epoch_num
        self.clsf_rpt = clsf_rpt

        


In [ ]:
# get the label from any image path  @TODO sloooooowwwwww......

def get_label(o):
    fname = o.name
    if fname in df.image_id.values:
        return df[df.image_id == fname].label.values[0]
    if fname in ensemble_train_df.image_id.values:
        return ensemble_train_df[ensemble_train_df.image_id == fname].label.values[0]


In [ ]:
import sys
sys.path.append('../utils')

from fastbook import *
from fastai.vision.all import *
from fastai.vision.widgets import *
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from string import ascii_uppercase
import numpy as np
import operator
import glob

import torch
from torch import nn as nn
from torch.nn import functional as F

import gc
# from cassava_utils_eff import *
from nn_utils_eff import *
# from pip_augs import *
from augmentation import *

%matplotlib inline
# %matplotlib widget

plt.rcParams['figure.figsize'] = [9, 5]
plt.rcParams['figure.dpi'] = 120
plt.style.use('dark_background')

import warnings
warnings.filterwarnings('ignore')  # or 'once'  


data_path = Path('../data')


In [ ]:
import pyforest

from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, ConfusionMatrixDisplay

RUN_NAME = '20210329-1845 - arch=tf_efficientnet_b4_ns - samples=-1 frozen=1 epochs=50 bs=48 res=380'


learn = load_learner(f'../models/{RUN_NAME}.pkl', cpu=False)
#test_learn = learn
c = remove_cbs(learn)
learn.remove_cbs([ModelToHalf, NonNativeMixedPrecision])
with open(f'models/DLS/{RUN_NAME}-DLS.pkl', 'rb') as f:
    dls = pickle.load(f)
dls = dls.cuda()

for num in range(50):
    load_model(f'models/{RUN_NAME}_{num}.pth', learn, opt=ranger, with_opt=False)
    learn.dls = dls

    preds, targs = learn.tta(ds_idx=2, n=4)
#     matrix = confusion_matrix(targs, preds.argmax(dim=-1))

    print(f'\n\nEPOCH #{num} - {RUN_NAME}\n')
    matrix_plot = ConfusionMatrixDisplay(matrix, display_labels=classes)
    matrix_plot.plot(cmap='Blues')


    plt.show()
    print(classification_report(targs, preds.argmax(dim=-1),target_names=classes))

    print('------------')